In [1]:
import torch
import torch.nn as nn
from strokes import StrokePatientsMIDataset, StrokePatientsMIProcessedDataset
from strokesdict import STROKEPATIENTSMI_LOCATION_DICT
from torcheeg.transforms import Select,BandSignal,Compose
from to import ToGrid, ToTensor
from downsample import SetSamplingRate
from baseline import BaselineCorrection

dataset = StrokePatientsMIDataset(root_path='./subdataset',
                                #   io_path='.torcheeg/datasets_1742368567479_nLgAA',
                        chunk_size=500,  # 1 second
                        overlap = 250,
                        offline_transform=Compose(
                                [BaselineCorrection(),
                                SetSamplingRate(origin_sampling_rate=500,target_sampling_rate=128),
                                BandSignal(sampling_rate=128,band_dict={'frequency_range':[8,40]})
                                ]),
                        online_transform=Compose(
                                [ToGrid(STROKEPATIENTSMI_LOCATION_DICT),ToTensor()]),
                        label_transform=Select('label'),
                        num_worker=8
)
print(dataset[0][0].shape) #EEG shape:torch.Size([1, 128, 9, 9])
print(dataset[0][1])  # label (int)
print(len(dataset))

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2025-04-23 16:37:58] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to .torcheeg/datasets_1745397478782_OwHIj.
[2025-04-23 16:37:58] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]: 100%|██████████| 1/1 [00:00<00:00, 61.83it/s]

[RECORD ./subdataset/sourcedata/sub-01/sub-01_task-motor-imagery_eeg.mat]: 0it [00:00, ?it/s]
[RECORD ./subdataset/sourcedata/sub-01/sub-01_task-motor-imagery_eeg.mat]: 1it [00:00,  4.34it/s]
[RECORD ./subdataset/sourcedata/sub-01/sub-01_task-motor-imagery_eeg.mat]: 2it [00:00,  5.48it/s]
[RECORD ./subdataset/sourcedata/su

torch.Size([1, 128, 9, 9])
0
240


In [2]:
from eegswintransformer import SwinTransformer

HYPERPARAMETERS = {
    "seed": 42,
    "batch_size": 16,
    "lr": 1e-4,
    "weight_decay": 1e-4,
    "num_epochs": 50,
}
from torcheeg.model_selection import KFoldPerSubjectGroupbyTrial
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader
from classifier import ClassifierTrainer

k_fold = KFoldPerSubjectGroupbyTrial(
    n_splits=4,
    shuffle=True,
    # split_path='.torcheeg/model_selection_1741918660674_jp0WB',
    random_state=42)

training_metrics = []
test_metrics = []

for i, (training_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
    model = SwinTransformer(patch_size=(8,3,3),
                              num_classes=2,
                              depths=(2, 6, 4),
                              num_heads=(3,6,8),
                              window_size=(3,3,3),
                              in_chans=1
                              ) # T, W, H 同时缩小
    trainer = ClassifierTrainer(model=model,
                                num_classes=2,
                                lr=HYPERPARAMETERS['lr'],
                                weight_decay=HYPERPARAMETERS['weight_decay'],
                                metrics=["accuracy"],
                                accelerator="gpu")
    training_loader = DataLoader(training_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=True)
    test_loader = DataLoader(test_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=False)
    # 提前停止回调
    early_stopping_callback = EarlyStopping(
        monitor='train_loss',
        patience=50,
        mode='min',
        verbose=True
    )
    trainer.fit(training_loader,
                test_loader,
                max_epochs=HYPERPARAMETERS['num_epochs'],
                callbacks=[early_stopping_callback],
                # enable_progress_bar=True,
                enable_model_summary=False,
                limit_val_batches=0.0)
    training_result = trainer.test(training_loader,
                                   enable_progress_bar=True,
                                   enable_model_summary=True)[0]
    test_result = trainer.test(test_loader,
                               enable_progress_bar=True,
                               enable_model_summary=True)[0]
    training_metrics.append(training_result["test_accuracy"])
    test_metrics.append(test_result["test_accuracy"])
    
     

[2025-04-23 16:38:13] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2025-04-23 16:38:13] INFO (torcheeg/MainThread) 😊 | Please set split_path to .torcheeg/model_selection_1745397493985_gHmxW for the next run, if you want to use the same setting for the experiment.
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarni

Epoch 0: 100%|██████████| 10/10 [00:01<00:00,  7.85it/s, loss=1.84, train_loss=0.763, train_accuracy=0.562]

Metric train_loss improved. New best score: 0.763


Epoch 0: 100%|██████████| 10/10 [00:01<00:00,  7.84it/s, loss=1.84, train_loss=0.763, train_accuracy=0.562]

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_accuracy', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
[2025-04-23 16:38:19] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.839 train_accuracy: 0.538 



...

Testing DataLoader 0: 100%|██████████| 13/13 [00:00<00:00, 61.26it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy                 1.0
        test_loss          1.701698602118995e-05
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 71.87it/s]

[2025-04-23 16:40:47] INFO (torcheeg/MainThread) 
[Test] test_loss: 0.274 test_accuracy: 0.950 



Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 65.45it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy          0.949999988079071
        test_loss           0.2737709581851959
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [3]:
import numpy as np

for i in range(0, len(test_metrics), 4):
    print(f"{i}\t"
          f"{np.mean(training_metrics[i:i+4]):.3f}\t"
          f"{np.std(training_metrics[i:i+4]):.3f}\t"
          f"{np.mean(test_metrics[i:i+4]):.3f}\t"
          f"{np.std(test_metrics[i:i+4]):.3f}")


0	1.000	0.000	0.884	0.088


In [4]:
for i, score in enumerate(test_metrics):
    print(f"{score:.3f}", end="\t")
    if (i + 1) % 4 == 0:
        print()  # Print a newline every 4 elements

# Ensure the last line is printed properly if the length isn't a multiple of 4
if len(test_metrics) % 4 != 0:
    print()


0.750	0.863	0.975	0.950	


In [ ]:
eeg = dataset[0][0]
print(eeg.shape)

In [ ]:
channel = eeg[:,:,2,5]
print(channel.shape)

In [ ]:
import matplotlib.pyplot as plt

# 假设 `eeg` 是 (4, 128) 形状的 EEG 频带数据
eeg = channel

# 选择第一个频带 (索引 0)
band_idx = 0
eeg_band = eeg[band_idx]  # 形状变为 (128,)

# 绘制折线图
plt.figure(figsize=(10, 4))
plt.plot(eeg_band, label=f'Band {band_idx}')
plt.xlabel("Time Points")
plt.ylabel("Amplitude")
plt.title(f"EEG Time Series for Frequency Band {band_idx}")
plt.legend()
plt.grid()
plt.show()


In [ ]:
from eegswintransformer import SwinTransformer
import torch

x = torch.randn(2,1,128,9,9)
model = SwinTransformer(patch_size=(8,3,3),
                        num_classes=2,
                        depths=(2, 6, 4),
                        num_heads=(3,6,8),
                        window_size=(3,3,3),
                        in_chans=1
                        ) # T, W, H 同时缩小
y = model(x)
print(y.shape)



In [ ]:
# 例如，选择 (3,4), (4,4), (4,5), (5,4) 作为重要通道（模拟 C3, C4, F3, F4）
focus_channels = [(3,4), (4,4), (4,5), (5,4)] 
from eca import ECA
import torch
# 初始化 ECA 并设置关注通道
eca = ECA(num_attention_heads=4, seq_len=128, hidden_dropout_prob=0.1, focus_channels=focus_channels)

# 生成测试数据
input_tensor = torch.randn(2, 1, 128, 9, 9)

# 测试
output_tensor = eca(input_tensor)

print("输入形状:", input_tensor.shape)  # (2, 1, 128, 9, 9)
print("输出形状:", output_tensor.shape)  # (2, 1, 128, 9, 9)


In [ ]:
import scipy.io

# Load the .mat file
mat_data = scipy.io.loadmat('./subdataset/sourcedata/sub-46/sub-46_task-motor-imagery_eeg.mat')

# Print all the keys in the loaded dictionary
print(mat_data.keys())

# Access specific data
# For example, if your .mat file contains a variable named 'data'
data = mat_data['__globals__']

# Print the shape and content of the data
print(data)
